In [1]:
import aisisax.object_detection.lsa_interface as aisax_object_detection
import aisisax.llm.openai_connector as aisax_openai
import aisisax.llm.ollama_connector as aisax_ollama
import cv2
import json

from IPython.display import display

/Users/haraldkriebisch/Development/sisax-hackathon-skafra/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
pImage = "assets/car.jpeg"

In [3]:
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")
img_counter = 0

# Speichert den bisherigen Spielablauf aus sicht des dritten Spielers
game_dic = {
    "initial_hand_cards": [],
    "player_of_first_turn": "",
    "maximum_bidding_value": 0,
    "bidding_winner": "",
    
}


In [4]:
def getImage():
    img_name = "assets/opencv_frame_{}.png".format(img_counter)
    cv2.imwrite(img_name, frame)
    print("{} written!".format(img_name))

    # See if a skat card was found and show the resulting image detection
    res = aisax_object_detection.call_lsa(img_name, "card")
    display(res)
    return img_name

In [5]:
def getInitialPlayingCards():
    #img = getImage()
    img = "assets/opencv_frame_0.png"

    openai_result = aisax_openai.generate_multimodal_answer("""
Welche Skatkarten kannst du erkennen? returne ein array of objects wobei jedes objekt eine karte repräsentiert. 
jedes objekt hat zwei attribute "symbol" and "kind".

"kind" können ausschließlich items aus der folgenden liste sein: ["sieben", "acht", "neun", "zehn", "bube", "Dame", "könig", "ass"];
"symbol" können ausschließlich items aus der folgenden liste sein: ["kreuz", "blatt", "herz", "karo"]

Der Output sollte sein, e.g. 
[
{
    symbol: "herz",
    kind: "sieben"
}
]
""", image_path=img)
    # Step 1: Remove the wrapping ```json and ``` characters
    clean_json = openai_result.strip("```json").strip("```")

    # Step 2: Parse the cleaned string into a Python list
    result_list = json.loads(clean_json)
    # TODO type umwandeln, sollte ein array sein.
    game_dic["initial_hand_cards"] = result_list 
    print(result_list)

In [6]:
def getPlayerOfFirstTurn():
    playerOfFirstTurn = input("Who is playing the first turn? (p1, p2, com) (sequence is always ... -> p1 -> p2 -> com -> p1 -> ...)")
    print(f"Alright so {playerOfFirstTurn} will be going first this round!")
    game_dic["player_of_first_turn"] = playerOfFirstTurn

In [7]:
def isFirstPlayer():
    return game_dic["player_of_first_turn"] == "com"

In [8]:
# Maximalen Reizwert erfragen
def getMaximumBiddingValue():
    openai_result = aisax_openai.generate_answer(f"""

    
Wie weit kann ich reizen? Gib mir einen Integer zurück
Hier sind die meine Hand-Karten: {game_dic["initial_hand_cards"]}

Hier sind die reiz-regeln:

Das Berechnen des Reizwerts
Die Rechnung für den Reizwert hat mehrere Faktoren:

Wie viele Buben habe ich bzw. habe ich nicht? Noch genauer: Wie viele Buben habe ich fortlaufend bzw. wie viele Buben fehlen mir fortlaufend?

Mit "fortlaufend" ist immer die Reihe gemeint, in der die Karten der Stärke nach sortiert werden. Also vom Kreuz-Buben, über den Pik-Buben, Herz-Buben, Karo-Buben.
Habe ich alle vier Buben, so zählt man "mit 4". Habe ich keinen Buben, so zählt man "ohne 4".
Habe ich nur den Kreuz-Buben "mit 1", habe ich alle anderen Buben außer dem Kreuz-Buben "ohne 1".
Soweit ganz einfach, jetzt eine kleine Falle: ist die Reihe unterbrochen, wird nur bis zum fehlenden Buben gezählt: hat man z.B. alle Buben außer dem Pik-Buben, hat man "mit 1", weil nach dem ersten Buben die Reihe unterbrochen ist. Hat man alle Buben bis auf den Herz-Buben, hat man "mit 2", hat man nur den Herz-Buben, dann "ohne 2".

Dafür, dass man bereit ist zu spielen, bekommt man einen Punkt. Das nennt man beim Reizen "Spiel" und rechnet "+1".
Hat man also seine Buben-Reihe gezählt und hat z.B. "mit 2" (den Kreuz-Buben und den Pik-Buben, aber keinen Herz-Buben), rechnet man einfach "mit 2" und "+1". Der Skatspieler zählt also seine zwei fortlaufenden Buben und addiert die "+1", weil er spielen möchte und sagt folgendes: "Mit zwei Spiel drei".

Jetzt ist es fast geschafft. Den bisher errechneten Wert (fortlaufende Buben/fortlaufend fehlende Buben + 1) multipliziert man mit dem Wert, den ein Spiel hat. Da dieser Wert einfach irgendwann einmal festgelegt wurde, lernt man ihn entweder auswendig oder schaut jedes Mal nach:
Farbspiel Karo: 9 | Farbspiel Herz: 10 | Farbspiel Pik: 11 | Farbspiel Kreuz 12 | Grand: 24

Jetzt rechnen wir in einem Beispiel alles zusammen:

Kreuz-Bube und Pik-Bube: Mit 2 = 2
Ich würde spielen, also +1: Mit 2 Spiel 3 = 3
Ich wähle ein Herz-Farbspiel (Wert/Faktor 10): Mit 2 Spiel 3 × 10 = 30
Der Reizwert beträgt folglich "30".

GIB MIR NUR DIE ZAHL ZURÜCK OHNE JEGLICHE ERKLÄRUNG, FOLGLICH MAXIMAL DREI ZEICHEN!!!!
"""
                                                )
    # TODO type umwandeln, sollte ein integer sein
    game_dic["maximum_bidding_value"] = openai_result 
    print(f"Ich werde bis maximal {game_dic['maximum_bidding_value']} reizen.")

In [9]:
def getBiddingWinner():
    biddingWinner = input("Who won the bidding? (p1, p2, com)")
    print(f"Alright so {biddingWinner} won the bidding, got it!")
    game_dic["bidding_winner"] = biddingWinner

In [10]:
def getGameContract():
    if( game_dic["bidding_winner"] == "com"):
    
        print("Hmmm... let's think...")
        # TODO: Currently we are cool and always playing "hand" so we don't look into the skat. We are so confident!
        print("We are playing 'hand' for sure!")
    
        openai_result = aisax_openai.generate_answer(f"""
    Welches Spiel sollte ich hier am besten spielen?
    Erwarte Rückgabewert: "herz", "null", "blatt", "grand", ...

    my cards: {game_dic["initial_hand_cards"]}
    """)
        game_dic["game_contract"] = openai_result

    else:
        game_dic["game_contract"] = input("Was spielen wir denn? (null, grand, karo, herz, pik, kreuz)")


In [ ]:
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed so we got a new image
        if(len(game_dic["initial_hand_cards"]) == 0):
            getInitialPlayingCards()
            getPlayerOfFirstTurn()

            getMaximumBiddingValue()
            getBiddingWinner()
            getGameContract()

            if(isFirstPlayer()):
                print("do first turn")
                
            continue

        
        
        img_counter += 1

cam.release()
cv2.destroyAllWindows()

2024-11-22 15:59:43.848 python[88195:1904312] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-22 15:59:43.848 python[88195:1904312] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


[{'symbol': 'herz', 'kind': 'könig'}, {'symbol': 'karo', 'kind': 'Dame'}, {'symbol': 'kreuz', 'kind': 'könig'}, {'symbol': 'blatt', 'kind': 'sieben'}, {'symbol': 'herz', 'kind': 'sieben'}, {'symbol': 'blatt', 'kind': 'Dame'}, {'symbol': 'herz', 'kind': 'könig'}, {'symbol': 'blatt', 'kind': 'könig'}, {'symbol': 'herz', 'kind': 'zehn'}, {'symbol': 'blatt', 'kind': 'zehn'}]


Who is playing the first turn? (p1, p2, com) (sequence is always ... -> p1 -> p2 -> com -> p1 -> ...) p1


Alright so p1 will be going first this round!
Ich werde bis maximal 0 reizen.


In [ ]:
# Object detection

In [ ]:
# res = aisax_object_detection.call_lsa(pImage, "skatkarten")

In [ ]:
# display(res)

# Interaktion mit LLMs
In den folgenden Zellen wird der Code zur Interaktion mit multi-modalen LLMs gezeigt. Wir nutzen sowohl GPT4o für einfache Textanfragen, sowie für multi-modale Text-Bild-Anfragen. Im Anschluss wird die gleiche Funktionalität noch für ein selbst-gehostetes open-source LLM bei Cloud&Heat per ollama gezeigt.

## OpenAI GPT-4o

In [ ]:
# aisax_openai.generate_answer("Warum ist der Himmel blau?")

In [ ]:
# aisax_openai.generate_multimodal_answer("Beschreibe das Bild", image_path=pImage)

## LLAMA3.1 90B via OLLAMA @ CLOUD&HEAT